In [8]:
import pandas as pd
from pymongo import MongoClient

# Connect to the MongoDB instance
client = MongoClient('mongodb://localhost:27017/')


In [9]:
# Access the uk_food database
db = client['uk_food']


In [10]:
# Which establishments have a hygiene score equal to 20?
hygiene_20 = db.establishments.find({"Hygiene": 20})
num_hygiene_20 = hygiene_20.count_documents()
pprint.pprint(hygiene_20[0])
 

AttributeError: 'Cursor' object has no attribute 'count_documents'

In [11]:
df_hygiene_20 = pd.DataFrame(list(hygiene_20))
print("Number of establishments with hygiene score 20:", num_hygiene_20)
print("Number of rows in the DataFrame:", len(df_hygiene_20))
print(df_hygiene_20.head(10))


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 662ffa5f727a6dda5d6bae5c, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:

# Which establishments in London have a RatingValue greater than or equal to 4?
london_high_ratings = db.establishments.find({"LocalAuthority": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}})
num_london_high_ratings = london_high_ratings.count()
pprint.pprint(london_high_ratings[0])


In [ ]:
df_london_high_ratings = pd.DataFrame(list(london_high_ratings))
print("Number of establishments in London with RatingValue >= 4:", num_london_high_ratings)
print("Number of rows in the DataFrame:", len(df_london_high_ratings))
print(df_london_high_ratings.head(10))


In [ ]:
# What are the top 5 establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to the new restaurant "Penang Flavours"?
penang_flavours_lat = db.establishments.find_one({"BusinessName": "Penang Flavours"})["Latitude"]
penang_flavours_lon = db.establishments.find_one({"BusinessName": "Penang Flavours"})["Longitude"]

top_5_rating_5 = db.establishments.find({"RatingValue": 5}).sort([("Hygiene", 1)]).limit(5)
pprint.pprint(top_5_rating_5[0])

df_top_5_rating_5 = pd.DataFrame(list(top_5_rating_5))
print("Number of rows in the DataFrame:", len(df_top_5_rating_5))
print(df_top_5_rating_5.head(10))


In [ ]:

# How many establishments in each Local Authority area have a hygiene score of 0? Sort the results from highest to lowest, and print out the top ten local authority areas.
pipeline = [
    {"$match": {"Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthority", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

hygiene_0_by_local_authority = db.establishments.aggregate(pipeline)
df_hygiene_0 = pd.DataFrame(list(hygiene_0_by_local_authority))
df_hygiene_0.rename(columns={"_id": "LocalAuthority"}, inplace=True)
print(df_hygiene_0.head(5))